## CLT-Based Confidence Intervals Can Fail

In many cases, one can successfully use the Central Limit Theorem (CLT)
to compute a confidence interval or to build an automatic Monte Carlo
algorithm.  See *OptionPricingMeanMC_CLT* for an example.  However, an
analysis based on the CLT is heuristic, not rigorous.  This example
points to how CLT based confidence intervals might fail.


In [53]:
# Import necessary packages
import numpy as np
from scipy.stats import norm

# The test case

Consider the case of $Y = X^p$ where $X \sim \mathcal{U}(0,1)$.  Then
we can compute that 

\begin{gather*}
\mu = \int_0^1 x^p \, {\rm d} x  = \frac{1}{p+1}, \qquad \text{provided }
p > -1, \\
\mathbb{E}(Y^2) = \int_0^1 x^{2p} \, {\rm d} x  = \frac{1}{2p+1}, \qquad \text{provided }
p > -1/2, \\
\text{var}(Y) = \mathbb{E}(Y^2) - \mu^2  = \frac{p^2}{(2p+1)(p+1)^2}, \qquad \text{provided }
p > -1/2.
\end{gather*}


# Set up parameters
Now we try using *meanMC_CLT* on this test case

In [54]:
absTol = 0.01 #relative error tolerance
relTol = 0 # relative error tolerance
alpha = 0.01 #uncertainty
Ntry = 5000 #number of trials
Y = lambda n,p: np.power(np.random.rand(n,1),p) #Y=X^p where X is standard uniform
mu =lambda p: 1/(p+1) #true answer
muhat = np.zeros((Ntry,1))

### Run the simulation for a nice $p$

In [59]:

p = 0.4; # should be >-1 for mu to be finite, and >-0.5 for var(Y) to be finite

for j  in  range(Ntry): # perform Monte Carlo Ntry times
    muhat[j], nSamples =meanMC_CLT(lambda n:Y(n,p),absTol,relTol,alpha) #estimated mu using CLT confidence intervals

err = abs(mu(p)-muhat) # compute true error
fail = np.mean(err>absTol) #proportion of failures to meet tolerance

In [61]:
print("For Y = X.^", p, " with X distributed uniformly on [0, 1]")
print("For an uncertainty = ", 100*alpha, "%, with 1000 initial sample size and 1.2 as the inflation factor")
print("The absolute error tolerence is ", absTol, ' the true mean = ', mu(p) )
print("The CLT-based confidence interval fails ",100*fail, "% of the time for",Ntry,"trials")

For Y = X.^ 0.4  with X distributed uniformly on [0, 1]
For an uncertainty =  1.0 %, with 1000 initial sample size and 1.2 as the inflation factor
The absolute error tolerence is  0.01  the true mean =  0.7142857142857143
The CLT-based confidence interval fails  0.16 % of the time for 5000 trials


The above case works well.
### Run the simulation again for a bad $p$.

In [62]:
p = - 0.4; # should be >-1 for mu to be finite, and >-0.5 for var(Y) to be finite

for j  in  range(Ntry): # perform Monte Carlo Ntry times
    muhat[j], nSamples =meanMC_CLT(lambda n:Y(n,p),absTol,relTol,alpha) #estimated mu using CLT confidence intervals

err = abs(mu(p)-muhat) # compute true error
fail = np.mean(err>absTol) #proportion of failures to meet tolerance
print("For Y = X.^", p, " with X distributed uniformly on [0, 1]")
print("For an uncertainty = ", 100*alpha, "%, with 1000 initial sample size and 1.2 as the inflation factor")
print("The absolute error tolerence is ", absTol, ' the true mean = ', mu(p) )
print("The CLT-based confidence interval fails ",100*fail, "% of the time for",Ntry,"trials")

For Y = X.^ -0.4  with X distributed uniformly on [0, 1]
For an uncertainty =  1.0 %, with 1000 initial sample size and 1.2 as the inflation factor
The absolute error tolerence is  0.01  the true mean =  1.6666666666666667
The CLT-based confidence interval fails  2.04 % of the time for 5000 trials


In this case the algorithm fails more than 1% of the time because the
variance estimates are not accurate.  One can check that the variance of
the variance is infinite.

In [58]:
def meanMC_CLT(inputRandomFuc,absTol,relTol,alpha): #Simple version of meanMC_CLT
    nsig = 1000 # initial size of the sample
    inflate = 1.2 # inflation rate
    YY = inputRandomFuc(nsig)
    sigma  = np.std(YY,ddof = 1)
    hum = np.mean(YY)
    sigmaUpBound = sigma * inflate #upper ound on the standard deviation
    nmu = max(1, np.power(np.ceil(norm.ppf(1-alpha/2)*sigmaUpBound/max(absTol,relTol)),2).astype(int) )  # number of samples needed for the error tolerance
    mu = np.mean(inputRandomFuc(nmu))
    nSample = nsig + nmu
    return mu, nSample